<a href="https://colab.research.google.com/github/IsaacIndex/ISOM3260-CourseRegistration/blob/main/Isom3260.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -- Install and setup Oracle Instant Client --
! wget -O oracleinstantclient.zip https://download.oracle.com/otn_software/linux/instantclient/19600/instantclient-basic-linux.x64-19.6.0.0.0dbru.zip
! unzip -oq oracleinstantclient.zip
! mkdir -p /opt/oracle
! mv instantclient_19_6 /opt/oracle/
! sh -c "echo /opt/oracle/instantclient_19_6 > /etc/ld.so.conf.d/oracle-instantclient.conf"
! ln -s /usr/local/lib/python3.7/dist-packages/ideep4py/lib/libmkldnn.so.0 /usr/local/lib/python3.7/dist-packages/ideep4py/lib/libmkldnn.0.so
! ldconfig


# -- Include cx_Oracle package --
! pip install cx_Oracle


--2023-03-30 07:11:10--  https://download.oracle.com/otn_software/linux/instantclient/19600/instantclient-basic-linux.x64-19.6.0.0.0dbru.zip
Resolving download.oracle.com (download.oracle.com)... 104.97.44.84
Connecting to download.oracle.com (download.oracle.com)|104.97.44.84|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75462547 (72M) [application/zip]
Saving to: ‘oracleinstantclient.zip’

oracleinstantclient 100%[===================>]  71.97M   179MB/s    in 0.4s    

2023-03-30 07:11:11 (179 MB/s) - ‘oracleinstantclient.zip’ saved [75462547/75462547]

ln: failed to create symbolic link '/usr/local/lib/python3.7/dist-packages/ideep4py/lib/libmkldnn.0.so': No such file or directory
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 888.2/888.2 KB 14.1 MB/s eta 0:00:00


In [ ]:
# Import cx_Oracle package and connect to Oracle Database
import cx_Oracle

HOST_NAME = "imz409.ust.hk"
PORT_NUMBER = "1521"
SERVICE_NAME = "imz409"
USERNAME = "nhwongaa"
PASSWORD = "2392"

dsn_tns = cx_Oracle.makedsn(HOST_NAME, PORT_NUMBER, service_name=SERVICE_NAME) 
conn = cx_Oracle.connect(user=USERNAME, password=PASSWORD, dsn=dsn_tns) 

c = conn.cursor()

In [ ]:
from google.colab.widgets import TabBar
import ipywidgets as widgets
import time
import pandas as pd
from ipywidgets import interact
from ipywidgets import Text
from ipywidgets import DatePicker
from ipywidgets import IntText
from ipywidgets import Label
from datetime import date

In [ ]:
#MANAGER

tb_login = TabBar(['Login'])
tb_main = TabBar(['Main Page', 'Insert Student Data', 'Update Student Data', 'Delete Student Data', 'Insert Course Data', 'Update Course Data', "Delete Course Data"])

# a simple function for jumping between tabs
# only an interaction with UI instead of the database
def showPage(tabName):
  print(tabName)
  if(tabName in tb_login.tab_names):
    display()
    with tb_login.output_to(tabName): 
      display()
  else:
    with tb_main.output_to(tabName): 
      display()


#########################################################
################ Login - Begin ################
#########################################################
with tb_login.output_to('Login'):

  # widgets
  Login_Login_Button = widgets.Button(description="Login")

  # login button clicked
  def Login_LoginAsStudent(b):
    showPage("Main Page")
  Login_Login_Button.on_click(Login_LoginAsStudent)
  display(Login_Login_Button)
################ Login - End ################


#######################################################
################ Main Page Tab - Begin ################
#######################################################
with tb_main.output_to('Main Page'):

# create 4 label 
  mainPage_label = Label("Please select a function...")
  mainPageStu_label = Label("STUDENT")
  UPD_spacer = Label("---------------------------------------------------------------------------------------------------------------------------------")
  mainPageCou_label = Label("COURSE")

# create 6 button
  btnInsertStuPage = widgets.Button(description="Insert Data")
  btnUpdateStuPage = widgets.Button(description="Update Data")
  btnDeleteStuPage = widgets.Button(description="Delete Data")
  btnInsertCouPage = widgets.Button(description="Insert Data")
  btnUpdateCouPage = widgets.Button(description="Update Data")
  btnDeleteCouPage = widgets.Button(description="Delete Data")

# define the event handlers for the two buttons when they are being clicked
  def on_btnInsertStuPage_button_clicked(b):
    showPage("Insert Student Data")
  btnInsertStuPage.on_click(on_btnInsertStuPage_button_clicked)

  def on_btnUpdateStuPage_button_clicked(b):
    showPage("Update Student Data")
  btnUpdateStuPage.on_click(on_btnUpdateStuPage_button_clicked)

  def on_btnDeleteStuPage_button_clicked(b):
    showPage("Delete Student Data")
  btnDeleteStuPage.on_click(on_btnDeleteStuPage_button_clicked)

  def on_btnInsertCouPage_button_clicked(b):
    showPage("Insert Course Data")
  btnInsertCouPage.on_click(on_btnInsertCouPage_button_clicked)

  def on_btnUpdateCouPage_button_clicked(b):
    showPage("Update Course Data")
  btnUpdateCouPage.on_click(on_btnUpdateCouPage_button_clicked)

  def on_btnDeleteCouPage_button_clicked(b):
    showPage("Delete Course Data")
  btnDeleteCouPage.on_click(on_btnDeleteCouPage_button_clicked)

# display the objects
  MainPage_row1HBox = widgets.HBox([mainPage_label])
  MainPage_row2HBox = widgets.HBox([mainPageStu_label])
  MainPage_row3HBox = widgets.HBox([btnInsertStuPage])
  MainPage_row4HBox = widgets.HBox([btnUpdateStuPage])
  MainPage_row5HBox = widgets.HBox([btnDeleteStuPage])
  MainPage_row6HBox = widgets.HBox([UPD_spacer])
  MainPage_row7HBox = widgets.HBox([mainPageCou_label])
  MainPage_row8HBox = widgets.HBox([btnInsertCouPage])
  MainPage_row9HBox = widgets.HBox([btnUpdateCouPage])
  MainPage_row10HBox = widgets.HBox([btnDeleteCouPage])


  display(widgets.VBox([MainPage_row1HBox, MainPage_row2HBox, MainPage_row3HBox, MainPage_row4HBox, MainPage_row5HBox, MainPage_row6HBox,MainPage_row7HBox, MainPage_row8HBox, MainPage_row9HBox, MainPage_row10HBox]))
################ Main Menu Tab - End ################


#########################################################
################ Insert Student Data Tab - Begin ################
#########################################################
with tb_main.output_to('Insert Student Data'):
  INS_stu_id_label = Label("Student ID:")
  INS_stu_id_textbox = Text(value='', placeholder='Enter Student ID', disabled=False)
  
  INS_stu_name_label = Label("Name:")
  INS_stu_name_textbox = Text(placeholder='Name', disabled=False)

  INS_stu_address_label = Label("Address:")
  INS_stu_address_textbox = Text(placeholder='Address', disabled=False)

  INS_stu_email_label = Label("Email:")
  INS_stu_email_textbox = Text(placeholder='Email', disabled=False)
 
  INS_stu_password_label = Label("Password:")
  INS_stu_password_textbox = Text(placeholder='Password', disabled=False)
 
  INS_stu_current_job_label = Label("Current Job:")
  INS_stu_current_job_textbox = Text(placeholder='Current Job', disabled=False)
 
  INS_stu_qualification_label = Label("Student ID:")
  INS_stu_qualification_textbox = Text(placeholder='Qualification', disabled=False)

  INS_add_button = widgets.Button(description="Add Student")
  INS_clear_button = widgets.Button(description="Clear")
  INS_back_button = widgets.Button(description="Back")

  def on_INS_add_button_clicked(b):
    c.execute(
    """INSERT INTO STUDENT VALUES ('{}', '{}', '{}', '{}', '{}', '{}', '{}')
    """.format(INS_stu_id_textbox.value, INS_stu_name_textbox.value, INS_stu_address_textbox.value, INS_stu_email_textbox.value, INS_stu_password_textbox.value, INS_stu_current_job_textbox.value, INS_stu_qualification_textbox.value))

    c.execute("commit")
    INS_add_button.disabled = True
    print("Done!")
  INS_add_button.on_click(on_INS_add_button_clicked)

  def on_INS_clear_button_clicked(b):
    INS_stu_id_textbox.value = ""
    INS_stu_name_textbox.value = ""
    INS_stu_address_textbox.value = ""
    INS_stu_email_textbox.value = ""
    INS_stu_password_textbox.value = ""
    INS_stu_current_job_textbox.value = ""
    INS_stu_qualification_textbox.value = ""
    INS_add_button.disabled = False
  INS_clear_button.on_click(on_INS_clear_button_clicked)

  def on_INS_back_button_clicked(b):
    showPage("Main Menu")
  INS_back_button.on_click(on_INS_back_button_clicked)

# display the objects
  INS_labelVBox = widgets.VBox([INS_stu_id_label, INS_stu_name_label, INS_stu_address_label, INS_stu_email_label, INS_stu_password_label, INS_stu_current_job_label, INS_stu_qualification_label])
  INS_textboxVBox = widgets.VBox([INS_stu_id_textbox, INS_stu_name_textbox, INS_stu_address_textbox, INS_stu_email_textbox, INS_stu_password_textbox, INS_stu_current_job_textbox, INS_stu_qualification_textbox])
  INS_line1HBox = widgets.HBox([INS_labelVBox, INS_textboxVBox])
  INS_line2HBox = widgets.HBox([INS_add_button, INS_clear_button])
  INS_line3HBox = widgets.HBox([INS_back_button])

  display(widgets.VBox([INS_line1HBox, INS_line2HBox, INS_line3HBox]))


################ Insert Student Data Tab - End ################


#########################################################
################ Insert Course Data Tab - Begin ################
#########################################################
with tb_main.output_to('Insert Course Data'):
  INS_crs_id_label = Label("Course ID:")
  INS_crs_id_textbox = Text(value='', placeholder='Enter Course ID', disabled=False)
  
  INS_crs_name_label = Label("Name:")
  INS_crs_name_textbox = Text(placeholder='Name', disabled=False)

  INS_crs_description_label = Label("Description:")
  INS_crs_description_textbox = Text(placeholder='Description', disabled=False)

  INS_crs_quota_label = Label("Quota:")
  INS_crs_quota_intbox = Text(placeholder='Quota', disabled=False)
 
  INS_crs_timeslot_label = Label("Timeslot:")
  INS_crs_timeslot_textbox = Text(placeholder='Timeslot', disabled=False)
 
  INS_crs_enrollment_fee_label = Label("Enrollment fee:")
  INS_crs_enrollment_fee_textbox = Text(placeholder='Enrollment fee', disabled=False)
 
  INS_crs_status_label = Label("Status:")
  INS_crs_status_textbox = Text(placeholder='Status', disabled=False)

  INS_add_button = widgets.Button(description="Add Course")
  INS_clear_button = widgets.Button(description="Clear")
  INS_back_button = widgets.Button(description="Back")

  def on_INS_add_button_clicked(b):
    c.execute(
    """INSERT INTO COURSE VALUES ('{}', '{}', '{}', {}, '{}', {}, '{}')
    """.format(INS_crs_id_textbox.value, INS_crs_name_textbox.value, INS_crs_description_textbox.value, INS_crs_quota_intbox.value, INS_crs_timeslot_textbox.value, INS_crs_enrollment_fee_textbox.value, INS_crs_status_textbox.value))

    c.execute("commit")
    INS_add_button.disabled = True
    print("Done!")
  INS_add_button.on_click(on_INS_add_button_clicked)

  def on_INS_clear_button_clicked(b):
    INS_crs_id_textbox.value = ""
    INS_crs_name_textbox.value = ""
    INS_crs_description_textbox.value = ""
    INS_crs_quota_intbox.value = ""
    INS_crs_timeslot_textbox.value = ""
    INS_crs_enrollment_fee_textbox.value = ""
    INS_crs_status_textbox.value = ""
    INS_add_button.disabled = False
  INS_clear_button.on_click(on_INS_clear_button_clicked)

  def on_INS_back_button_clicked(b):
    showPage("Main Menu")
  INS_back_button.on_click(on_INS_back_button_clicked)

# display the objects
  INS_labelVBox = widgets.VBox([INS_crs_id_label, INS_crs_name_label, INS_crs_description_label, INS_crs_quota_label, INS_crs_timeslot_label, INS_crs_enrollment_fee_label, INS_crs_status_label])
  INS_textboxVBox = widgets.VBox([INS_crs_id_textbox, INS_crs_name_textbox, INS_crs_description_textbox, INS_crs_quota_intbox, INS_crs_timeslot_textbox, INS_crs_enrollment_fee_textbox, INS_crs_status_textbox])
  INS_line1HBox = widgets.HBox([INS_labelVBox, INS_textboxVBox])
  INS_line2HBox = widgets.HBox([INS_add_button, INS_clear_button])
  INS_line3HBox = widgets.HBox([INS_back_button])

  display(widgets.VBox([INS_line1HBox, INS_line2HBox, INS_line3HBox]))
################ Insert Course Data Tab - End ################


#########################################################
################ Update Student Data Tab - Begin ################
#########################################################
with tb_main.output_to('Update Student Data'):

# create 6 label and 5 textboxes
  UPD_stu_id_label = Label("Student ID: ")
  UPD_stu_id_textbox = Text(value='', placeholder='Student ID', disabled=False)

  UPD_spacer = Label("---------------------------------------------------------------------------------------------------------------------------------")

  UPD_stu_name_label = Label("Name: ")
  UPD_stu_name_textbox = Text(placeholder='Student Name', description='Name:', disabled=False)

  UPD_stu_dob_label = Label("DOB: ") #REAL DELETE??????
  UPD_stu_DOB_datepicker = DatePicker(description='Date of Birth: ', disabled=False) #REAL DELETE??????

  UPD_stu_email_label = Label("Email: ")
  UPD_stu_email_textbox = Text(description='Email: ', disabled=False)

  UPD_stu_address_label = Label("Address: ")
  UPD_stu_address_textbox = Text(description='Address: ', disabled=False)

  UPD_stu_password_label = Label("Password: ")
  UPD_stu_password_textbox = Text(description='Password: ', disabled=False)

  UPD_stu_currentjob_label = Label("Current Job: ")
  UPD_stu_currentjob_textbox = Text(description='Current Job: ', disabled=False)

  UPD_stu_qualification_label = Label("Qualification: ")
  UPD_stu_qualification_textbox = Text(description='Qualification: ', disabled=False)

# create 4 buttons
  UPD_retrieve_button = widgets.Button(description="Retrieve")
  UPD_update_button = widgets.Button(description="Update")
  UPD_clear_button = widgets.Button(description="Clear")
  UPD_back_button = widgets.Button(description="Back")

# define the event handlers for the three buttons when they are being clicked
  def on_UPD_retrieve_button_clicked(b):
    c.execute("SELECT * FROM STUDENT WHERE student_id = '" + UPD_stu_id_textbox.value + "'")
    for row in c:
      UPD_stu_name_textbox.value = row[1]
      UPD_stu_DOB_datepicker.value = row[2] #REAL DELETE??????
      UPD_stu_email_textbox.value = row[3]
      UPD_stu_address_textbox.value = row[4]
      UPD_stu_password_textbox.value = row[5]
      UPD_stu_currentjob_textbox.value = row[6]
      UPD_stu_qualification_textbox.value = row[7]
  UPD_retrieve_button.on_click(on_UPD_retrieve_button_clicked)
  
  def on_UPD_clear_button_clicked(b):
    UPD_stu_name_textbox.value = ""
    UPD_stu_DOB_datepicker.value = None #REAL DELETE??????
    UPD_stu_email_textbox.value = ""
    UPD_stu_address_textbox.value = ""
    UPD_stu_password_textbox.value = ""
    UPD_stu_currentjob_textbox.value = ""
    UPD_stu_qualification_textbox.value = ""
  UPD_clear_button.on_click(on_UPD_clear_button_clicked)  
    
  def on_UPD_update_button_clicked(b):
    c.execute(
    """
      UPDATE STUDENT 
      SET STU_NAME = '{}',
      STU_DOB = TO_DATE('{}', 'YYYY-MM-DD'), 
      STU_EMAIL = '{}',
      STU_ADDRESS = '{}',
      STU_PASSWORD = '{}',
      STU_CURRENT_JOB = '{}',
      STU_QUALIFICATION = '{}'
      WHERE STU_ID = '{}'
    """.format(UPD_stu_name_textbox.value, UPD_stu_DOB_datepicker.value.strftime("%Y-%m-%d"), UPD_stu_email_textbox.value, UPD_stu_address_textbox.value, UPD_stu_password_textbox.value, UPD_stu_currentjob_textbox.value, UPD_stu_qualification_textbox.value, UPD_stu_id_textbox.value)
    )
    c.execute("commit")
    print("Update Success!")
  UPD_update_button.on_click(on_UPD_update_button_clicked)

  def on_UPD_back_button_clicked(b):
    showPage("Main Menu")
  UPD_back_button.on_click(on_UPD_back_button_clicked)

# display the objects
  UPD_line1HBox = widgets.HBox([UPD_stu_id_label, UPD_stu_id_textbox, UPD_retrieve_button])
  UPD_line2HBox = widgets.HBox([UPD_spacer])
  UPD_textboxVBox = widgets.VBox([UPD_stu_name_textbox, UPD_stu_DOB_datepicker, UPD_stu_email_textbox, UPD_stu_address_textbox, UPD_stu_password_textbox, UPD_stu_currentjob_textbox, UPD_stu_qualification_textbox])
  UPD_line3HBox = widgets.HBox([UPD_textboxVBox])
  UPD_line4HBox = widgets.HBox([UPD_update_button, UPD_clear_button])
  UPD_line5HBox = widgets.HBox([UPD_back_button])
  
  display(widgets.VBox([UPD_line1HBox, UPD_line2HBox, UPD_line3HBox, UPD_line4HBox, UPD_line5HBox]))
################ Update Student Data Tab - End ################

#########################################################
################ Update Course Data Tab - Begin ################
#########################################################
with tb_main.output_to('Update Course Data'):

# create 6 label and 5 textboxes
  UPD_crs_id_label = Label("Course ID: ")
  UPD_crs_id_textbox = Text(value='', placeholder='Course ID', disabled=False)

  UPD_spacer = Label("---------------------------------------------------------------------------------------------------------------------------------")

  UPD_crs_name_label = Label("Course Name: ")
  UPD_crs_name_textbox = Text(placeholder='Course Name', description='Name:', disabled=False)

  UPD_crs_desc_label = Label("Course Description: ")
  UPD_crs_desc_textbox = Text(description='Description:', disabled=False)

  UPD_crs_quota_label = Label("Quota: ")
  UPD_crs_quota_intbox = IntText(description='Quota:', disabled=False)

  UPD_crs_timeslot_label = Label("Timeslot: ") ###### TIMESLOT NEED DATEPICKR?
  UPD_crs_timeslot_textbox = Text(description='Timeslot:', disabled=False) ###### TIMESLOT NEED DATEPICKR?

  UPD_crs_enrolfee_label = Label("Enrollment Fee: ") ###### ENROLLMENT FEE INT BOX?
  UPD_crs_enrolfee_intbox = IntText(description='Fee:', disabled=False) ####### ENROLLMENT FEE INT BOX?

  UPD_crs_status_label = Label("Status: ")
  UPD_crs_status_textbox = Text(description='Status:', disabled=False)

  UPD_crs_instructor_label = Label("Instructor ID: ")
  UPD_crs_instructor_textbox = Text(description='Instructor ID: ', disabled=False)

  
# create 4 buttons
  UPD_retrieve_button = widgets.Button(description="Retrieve")
  UPD_update_button = widgets.Button(description="Update")
  UPD_clear_button = widgets.Button(description="Clear")
  UPD_back_button = widgets.Button(description="Back")

# define the event handlers for the three buttons when they are being clicked
  def on_UPD_retrieve_button_clicked(b):
    c.execute("SELECT * FROM COURSE WHERE CRS_ID = '" + UPD_crs_id_textbox.value + "'")
    for row in c:
      UPD_crs_name_textbox.value = row[1]
      UPD_crs_desc_textbox.value = row[2]
      UPD_crs_quota_intbox.value = row[3]
      UPD_crs_timeslot_textbox.value = row[4] ####TIMESLOTTTTTTTT
      UPD_crs_enrolfee_intbox.value = row[5] ####### ENROLLMENT FEE INT BOX?
      UPD_crs_status_textbox.value = row[6]
      UPD_crs_instructor_textbox.value = row[7]
  UPD_retrieve_button.on_click(on_UPD_retrieve_button_clicked)
  
  def on_UPD_clear_button_clicked(b):
    UPD_crs_name_textbox.value = ""
    UPD_crs_desc_textbox.value = ""
    UPD_crs_quota_intbox.value = 0 
    UPD_crs_timeslot_textbox.value = "" ####TIMESLOTTTTTTTT
    UPD_crs_enrolfee_intbox.value = 0 ####### ENROLLMENT FEE INT BOX?
    UPD_crs_status_textbox.value = ""
    UPD_crs_instructor_textbox.value = ""
  UPD_clear_button.on_click(on_UPD_clear_button_clicked)  
    
  def on_UPD_update_button_clicked(b):
    c.execute(
    """
      UPDATE COURSE 
      SET CRS_NAME = '{}',
      CRS_DESC = '{}',
      CRS_QUOTA = {},
      CRS_TIMESLOT = '{}', 
      CRS_ENROLLMENT_FEE = {},
      CRS_STATUS = '{}',
      INSTRUCTOR_INS_ID ='{}',
      WHERE CRS_ID = '{}'
    """.format(UPD_crs_name_textbox.value, UPD_crs_desc_textbox.value, UPD_crs_quota_intbox.value, UPD_crs_timeslot_textbox.value, UPD_crs_enrolfee_intbox.value, UPD_crs_status_textbox.value, UPD_crs_instructor_textbox.value, UPD_crs_id_textbox.value))
    c.execute("commit")
    print("Update Success!")
  UPD_update_button.on_click(on_UPD_update_button_clicked)

  def on_UPD_back_button_clicked(b):
    showPage("Main Page")
  UPD_back_button.on_click(on_UPD_back_button_clicked)
  

# display the objects
  UPD_line1HBox = widgets.HBox([UPD_crs_id_label, UPD_crs_id_textbox, UPD_retrieve_button])
  UPD_line2HBox = widgets.HBox([UPD_spacer])
  UPD_textboxVBox = widgets.VBox([UPD_crs_name_textbox, UPD_crs_desc_textbox, UPD_crs_quota_intbox, UPD_crs_timeslot_textbox, UPD_crs_enrolfee_intbox, UPD_crs_status_textbox, UPD_crs_instructor_textbox])
  UPD_line3HBox = widgets.HBox([UPD_textboxVBox])
  UPD_line4HBox = widgets.HBox([UPD_update_button, UPD_clear_button])
  UPD_line5HBox = widgets.HBox([UPD_back_button])
  
  display(widgets.VBox([UPD_line1HBox, UPD_line2HBox, UPD_line3HBox, UPD_line4HBox, UPD_line5HBox]))
################ Update Course Data Tab - End ################

#########################################################
################  Delete Student Data Tab - Begin ################
#########################################################



################ Delete Student Data Tab - End ################


#####

####################################################
################  Delete Course Data Tab - Begin ################
#########################################################



################ Delete Course Data Tab - End ################

showPage("Login")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#STUDENT